<a href="https://colab.research.google.com/github/jainriya9/DeepFake-Detection/blob/main/pipeline_formation%2C_still_overfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.listdir('/content/drive/MyDrive/Celeb-DF'))

['List_of_testing_videos.txt', 'Celeb-synthesis', 'Celeb-real', 'YouTube-real', 'Frames-Real', 'Frames-Fake', 'Frames-Real-20240626T130257Z-001.zip', 'Frames-Fake-20240626T130258Z-001.zip', 'Frames-YouTube-real', 'model.h5', 'Split-Frames']


In [ ]:
# Define the paths to your dataset in Google Drive
real_videos_dir = '/content/drive/MyDrive/Celeb-DF/Celeb-real'
fake_videos_dir = '/content/drive/MyDrive/Celeb-DF/Celeb-synthesis'

# Verify that the paths are correct
import os
print(os.listdir(real_videos_dir))
print(os.listdir(fake_videos_dir))

['id0_0000.mp4', 'id0_0001.mp4', 'id0_0002.mp4', 'id0_0005.mp4', 'id0_0006.mp4', 'id10_0000.mp4', 'id0_0009.mp4', 'id0_0003.mp4', 'id0_0008.mp4', 'id0_0007.mp4', 'id0_0004.mp4', 'id11_0005.mp4', 'id10_0001.mp4', 'id10_0006.mp4', 'id11_0006.mp4', 'id10_0002.mp4', 'id10_0008.mp4', 'id10_0007.mp4', 'id10_0003.mp4', 'id10_0009.mp4', 'id10_0005.mp4', 'id11_0003.mp4', 'id11_0002.mp4', 'id11_0000.mp4', 'id11_0001.mp4', 'id10_0004.mp4', 'id11_0004.mp4', 'id12_0004.mp4', 'id13_0005.mp4', 'id13_0008.mp4', 'id13_0000.mp4', 'id11_0008.mp4', 'id13_0007.mp4', 'id13_0009.mp4', 'id13_0006.mp4', 'id11_0010.mp4', 'id12_0000.mp4', 'id12_0001.mp4', 'id13_0004.mp4', 'id11_0009.mp4', 'id13_0001.mp4', 'id12_0003.mp4', 'id11_0007.mp4', 'id12_0006.mp4', 'id13_0003.mp4', 'id12_0005.mp4', 'id13_0002.mp4', 'id12_0002.mp4', 'id13_0015.mp4', 'id13_0014.mp4', 'id16_0003.mp4', 'id13_0013.mp4', 'id13_0010.mp4', 'id16_0000.mp4', 'id13_0012.mp4', 'id16_0001.mp4', 'id16_0002.mp4', 'id13_0011.mp4', 'id17_0006.mp4', 'id16_

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2
# Output paths for frames
real_frames_path = '/content/drive/MyDrive/Celeb-DF/Frames-Real'
fake_frames_path = '/content/drive/MyDrive/Celeb-DF/Frames-Fake'

In [ ]:

# Define minimal data augmentation for real frames
datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True
)

# Augment frames without saving
def augment_frames(image_folder):
    augmented_frames = []
    for filename in os.listdir(image_folder):
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        image = image.reshape((1,) + image.shape)  # Reshape for datagen
        augmented_iter = datagen.flow(image, batch_size=1)
        augmented_frames.append(next(augmented_iter)[0].astype(np.uint8))  # Take one augmented frame
    return augmented_frames

# Load and augment real frames only
augmented_real_frames = augment_frames(real_frames_path)
print(f"Augmented {len(augmented_real_frames)} real frames.")

Augmented 2538 real frames.


In [ ]:
# def load_and_process_fake_frames(fake_frames_path, batch_size=32):
#     """Loads and processes fake frames in batches."""
#     filenames = os.listdir(fake_frames_path)
#     num_batches = (len(filenames) + batch_size - 1) // batch_size

#     for batch_idx in range(num_batches):
#         start_idx = batch_idx * batch_size
#         end_idx = min((batch_idx + 1) * batch_size, len(filenames))
#         batch_filenames = filenames[start_idx:end_idx]

#         batch_frames = []
#         for filename in batch_filenames:
#             image_path = os.path.join(fake_frames_path, filename)
#             image = cv2.imread(image_path)
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#             image = cv2.resize(image, (256, 256))
#             batch_frames.append(image)

#         # Convert batch to numpy array
#         batch_frames = np.array(batch_frames)

# # Call the function to load and process frames in batches
# load_and_process_fake_frames(fake_frames_path)

In [ ]:
# Load fake frames
fake_frames = [cv2.cvtColor(cv2.imread(os.path.join(fake_frames_path, filename)), cv2.COLOR_BGR2RGB) for filename in os.listdir(fake_frames_path)]
print(f"Loaded {len(fake_frames)} fake frames.")

Loaded 12888 fake frames.


In [ ]:
# Resize fake frames before converting to numpy array
fake_frames = [cv2.resize(frame, (256, 256)) for frame in fake_frames]

# Ensure all augmented real frames are the same size
augmented_real_frames = [cv2.resize(frame, (256, 256)) for frame in augmented_real_frames]

# Convert lists to numpy arrays
augmented_real_frames = np.array(augmented_real_frames)
fake_frames = np.array(fake_frames)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, LayerNormalization, Add, Input, Flatten, Dense
from tensorflow.keras.models import Model

# Define a simplified PVT block
def pvt_block(x, num_channels):
    shortcut = x
    # Initialize layers outside the function to avoid creating them during tracing
    conv1 = Conv2D(num_channels, kernel_size=1, strides=1, padding='same')
    norm1 = LayerNormalization()
    conv2 = Conv2D(num_channels, kernel_size=3, strides=1, padding='same')
    norm2 = LayerNormalization()
    proj = Conv2D(num_channels * 4, kernel_size=1, strides=1, padding='same')
    conv3 = Conv2D(num_channels * 4, kernel_size=1, strides=1, padding='same')
    norm3 = LayerNormalization()
    add = Add()

    x = conv1(x)  # Use the pre-initialized layers
    x = norm1(x)
    x = tf.nn.relu(x)
    x = conv2(x)
    x = norm2(x)
    x = tf.nn.relu(x)
    # Project the shortcut to match the number of channels
    shortcut = proj(shortcut)
    x = conv3(x)
    x = norm3(x)
    x = add([shortcut, x])
    x = tf.nn.relu(x)
    return x

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, LayerNormalization, Add, Input, Flatten, Dense # Import Flatten and Dense
from tensorflow.keras.models import Model

# Define the model
input_shape = (224, 224, 3)  # Adjust this based on your frame size
inputs = Input(shape=input_shape)
x = pvt_block(inputs, num_channels=8)
# Add more PVT blocks or layers as needed

x = Flatten()(x)
binary_output = Dense(1, activation='sigmoid', name='binary_output')(x)
distillation_output = Dense(1, activation='sigmoid', name='distillation_output')(x)
multimodal_output = Dense(1, activation='sigmoid', name='multimodal_output')(x)

modelrgb = Model(inputs, [binary_output, distillation_output, multimodal_output])

# Compile the model with multiple loss functions and metrics
modelrgb.compile(
    optimizer='adam',
    loss={'binary_output': 'binary_crossentropy', 'distillation_output': 'binary_crossentropy', 'multimodal_output': 'binary_crossentropy'},
    metrics={'binary_output': 'accuracy', 'distillation_output': 'accuracy', 'multimodal_output': 'accuracy'}
)

# Display the model summary
modelrgb.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 8)          32        ['input_1[0][0]']             
                                                                                                  
 layer_normalization (Layer  (None, 224, 224, 8)          16        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 tf.nn.relu (TFOpLambda)     (None, 224, 224, 8)          0         ['layer_normalization[0][0

In [ ]:
# Prepare the dataset
X = np.concatenate((augmented_real_frames, fake_frames))
# Resize all frames to match model input
X = np.array([cv2.resize(frame, (224, 224)) for frame in X])
y = np.concatenate((np.ones(len(augmented_real_frames)), np.zeros(len(fake_frames))))

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Train the model
modelrgb.fit(X_train, y_train, epochs=3, batch_size=16, validation_split=0.2)


Epoch 1/3
617/617 [==============================] - 160s 248ms/step - loss: 1060.4971 - binary_output_loss: 270.9855 - distillation_output_loss: 369.6587 - multimodal_output_loss: 419.8534 - binary_output_accuracy: 0.8156 - distillation_output_accuracy: 0.8065 - multimodal_output_accuracy: 0.8070 - val_loss: 126.6028 - val_binary_output_loss: 25.8935 - val_distillation_output_loss: 76.0532 - val_multimodal_output_loss: 24.6562 - val_binary_output_accuracy: 0.9088 - val_distillation_output_accuracy: 0.8744 - val_multimodal_output_accuracy: 0.9092
Epoch 2/3
617/617 [==============================] - 155s 251ms/step - loss: 48.5134 - binary_output_loss: 15.5819 - distillation_output_loss: 17.1664 - multimodal_output_loss: 15.7651 - binary_output_accuracy: 0.9109 - distillation_output_accuracy: 0.9131 - multimodal_output_accuracy: 0.9113 - val_loss: 22.7642 - val_binary_output_loss: 7.7471 - val_distillation_output_loss: 8.3607 - val_multimodal_output_loss: 6.6564 - val_binary_output_accu

In [ ]:
# Get predictions for training data
predictions_train = modelrgb.predict(X_train)
distillation_predictions_train = predictions_train[1] # Access the second element of the list, which corresponds to distillation output
multimodal_predictions_train = predictions_train[2]   # Access the third element, which corresponds to multimodal output

# Get predictions for testing data
predictions_test = modelrgb.predict(X_test)
distillation_predictions_test = predictions_test[1]
multimodal_predictions_test = predictions_test[2]

97/97 [==============================] - 20s 209ms/step


In [ ]:
# Assuming you have already split your data into X_test and y_test

# Evaluate the model on test data
accuracy = modelrgb.evaluate(X_test, y_test)

# Print the accuracy
print(f"Model accuracy: {accuracy[5] * 100:.2f}%")  # Assuming accuracy is the 5th metric in your model's metrics list

97/97 [==============================] - 20s 210ms/step - loss: 11.1751 - binary_output_loss: 2.8963 - distillation_output_loss: 4.1040 - multimodal_output_loss: 4.1748 - binary_output_accuracy: 0.9300 - distillation_output_accuracy: 0.9494 - multimodal_output_accuracy: 0.9417
Model accuracy: 94.94%


In [ ]:

# Evaluate the model on test data
accuracy = modelrgb.evaluate(X_test, y_test)

# Print the evaluation results
print(f"Model evaluation results:")
print(f"Loss: {accuracy[0]}")
print(f"Binary Output Accuracy: {accuracy[5] * 100:.2f}%")
print(f"Distillation Output Accuracy: {accuracy[6] * 100:.2f}%")
print(f"Multimodal Output Accuracy: {accuracy[6] * 100:.2f}%")

97/97 [==============================] - 20s 210ms/step - loss: 11.1751 - binary_output_loss: 2.8963 - distillation_output_loss: 4.1040 - multimodal_output_loss: 4.1748 - binary_output_accuracy: 0.9300 - distillation_output_accuracy: 0.9494 - multimodal_output_accuracy: 0.9417
Model evaluation results:
Loss: 11.175082206726074
Binary Output Accuracy: 94.94%
Distillation Output Accuracy: 94.17%
Multimodal Output Accuracy: 94.17%


# SECOND PIPELINE

In [ ]:
import librosa
import numpy as np
from scipy.fftpack import fft
from scipy.signal import butter, lfilter

def extract_mfcc_fft(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    signal = image_gray.mean(axis=0)

    # High-pass filter
    cutoff_freq = 100  # Adjust this value based on your desired cutoff frequency
    nyquist_freq = 0.5 * 22050
    normal_cutoff = cutoff_freq / nyquist_freq
    b, a = butter(4, normal_cutoff, btype='high', analog=False)
    filtered_signal = lfilter(b, a, signal)

    # Adjust n_fft based on the length of filtered_signal
    n_fft = min(2048, len(filtered_signal))  # Set n_fft to be smaller of 2048 or signal length
    mfccs = librosa.feature.mfcc(y=filtered_signal, sr=22050, n_mfcc=13, n_fft=n_fft)
    fft_features = np.abs(fft(filtered_signal, n=n_fft))
    return mfccs, fft_features

# Example usage
# real_features = [extract_mfcc_fft(frame) for frame in augmented_real_frames]
# fake_features = [extract_mfcc_fft(frame) for frame in fake_frames]

# Ensure to handle varying signal lengths appropriately in your actual application.


# Extract features from augmented real frames
real_features = [extract_mfcc_fft(frame) for frame in augmented_real_frames]

# Extract features from fake frames (not augmented)
fake_features = []
for filename in os.listdir(fake_frames_path):
    image_path = os.path.join(fake_frames_path, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mfcc, fft_features = extract_mfcc_fft(image)
    fake_features.append((mfcc, fft_features))

print(f"Extracted features from {len(real_features)} augmented real frames.")
print(f"Extracted features from {len(fake_features)} fake frames.")

/usr/local/lib/python3.10/dist-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


Extracted features from 2538 augmented real frames.
Extracted features from 12888 fake frames.


In [ ]:
import numpy as np

# Assuming you have extracted features stored in real_features and fake_features
# real_features and fake_features should be lists of tuples (mfccs, fft_features)

# Pad or truncate MFCC features to a consistent length
max_length_mfcc = max(len(f[0][0]) for f in real_features + fake_features)  # Find the maximum length
X_mfcc = []
for f in real_features + fake_features:
    mfcc = f[0]
    if len(mfcc[0]) < max_length_mfcc:
        # Pad with zeros along the time axis
        mfcc = np.pad(mfcc, ((0, 0), (0, max_length_mfcc - len(mfcc[0]))), mode='constant')
    elif len(mfcc[0]) > max_length_mfcc:
        # Truncate along the time axis
        mfcc = mfcc[:, :max_length_mfcc]
    X_mfcc.append(mfcc)

X_mfcc = np.array(X_mfcc)

# Pad or truncate FFT features to a consistent length
max_length_fft = max(len(f[1]) for f in real_features + fake_features)
X_fft = []
for f in real_features + fake_features:
    fft_features = f[1]
    if len(fft_features) < max_length_fft:
        # Pad with zeros
        fft_features = np.pad(fft_features, (0, max_length_fft - len(fft_features)), mode='constant')
    elif len(fft_features) > max_length_fft:
        # Truncate
        fft_features = fft_features[:max_length_fft]
    X_fft.append(fft_features)

X_fft = np.array(X_fft)

# Create labels: 0 for real, 1 for fake
y_labels = np.array([0] * len(real_features) + [1] * len(fake_features))

print(f"MFCC features shape: {X_mfcc.shape}")
print(f"FFT features shape: {X_fft.shape}")
print(f"Labels shape: {y_labels.shape}")

MFCC features shape: (15426, 13, 2)
FFT features shape: (15426, 974)
Labels shape: (15426,)


In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install vit-keras

In [ ]:
from tensorflow.keras.layers import Input
import numpy as np

# Assuming X_mfcc and X_fft are already defined
input_mfcc = Input(shape=X_mfcc.shape[1:], name='input_mfcc')
input_fft = Input(shape=X_fft.shape[1:], name='input_fft')

In [ ]:
!pip install --upgrade tensorflow
!pip install tensorflow tensorflow-addons vit-keras

In [ ]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Reshape, Add, MultiHeadAttention, Dropout, Concatenate, Conv1D
from tensorflow.keras.models import Model

# Define input shapes for MFCC and FFT features
input_mfcc_shape = (13, 2)  # MFCC shape: 13 coefficients, 2 time steps (assuming 2 channels)
input_fft_shape = (974,)  # FFT shape: 974 features

# Inputs for MFCC and FFT features
input_mfcc = Input(shape=input_mfcc_shape, name='input_mfcc')
input_fft = Input(shape=input_fft_shape, name='input_fft')

flattened_mfcc = Flatten()(input_mfcc)
flattened_fft = Flatten()(input_fft)

# Patch Embedding layer for MFCC and FFT
projection_dim = 256  # Adjust as needed
patch_mfcc = Dense(projection_dim)(input_mfcc)
patch_fft = Dense(projection_dim)(input_fft)


 #from here
# Reshape patch_fft to match the rank of patch_mfcc
patch_fft = Reshape((-1, projection_dim))(patch_fft) # Reshape patch_fft to be 3D

# Concatenate patches
combined_patches = tf.concat([patch_mfcc, patch_fft], axis=1)

# Reshape for PVT block
combined_patches = Reshape((-1, projection_dim))(combined_patches)

#to here

#new added from here

# Reshape patch_fft to match the rank of patch_mfcc
patch_fft = Reshape((-1, projection_dim))(patch_fft)

# Concatenate patches
combined_patches = tf.concat([patch_mfcc, patch_fft], axis=1)

# Reshape for PVT block
combined_patches = Reshape((-1, projection_dim))(combined_patches)

# Flatten combined_patches before passing to concatenate_1
flattened_combined_patches = Flatten()(combined_patches)

input_distillation_second = Input(shape=(distillation_predictions_train.shape[1],), name='input_distillation_second')
input_multimodal_second = Input(shape=(multimodal_predictions_train.shape[1],), name='input_multimodal_second')

# Now pass flattened_combined_patches to concatenate_1
concat_layer = Concatenate()([input_distillation_second,
                               input_multimodal_second,
                               flattened_combined_patches])

#to here

# Pyramid Vision Transformer (PVT) block
def pvt_block(x, num_channels):
    shortcut = x
    # Projection layers
    x = Dense(num_channels)(x)
    x = LayerNormalization()(x)
    x = tf.nn.relu(x)
    x = Dense(num_channels)(x)
    x = LayerNormalization()(x)
    x = tf.nn.relu(x)
    # Project the shortcut to match the number of channels
    shortcut = Dense(num_channels * 4)(shortcut)
    x = Dense(num_channels * 4)(x)
    x = LayerNormalization()(x)
    x = Add()([shortcut, x])
    x = tf.nn.relu(x)
    return x

# Apply PVT block
pvt_output = pvt_block(combined_patches, num_channels=256)  # Adjust num_channels as needed

# Vision Transformer (ViT) block
def vit_block(x, num_heads=8, ff_dim=512, dropout_rate=0.1):
    # Multi-head self-attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(attn_output + x)

    # Feed Forward Network
    ffn = Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(out1)
    # Project back to the original dimension
    ffn = Conv1D(filters=projection_dim, kernel_size=1)(ffn)
    ffn = Dropout(dropout_rate)(ffn)

    # Adjust the number of channels in the skip connection to match FFN output
    out1 = Conv1D(filters=projection_dim, kernel_size=1)(out1)

    out2 = LayerNormalization(epsilon=1e-6)(ffn + out1)
    return out2

vit_output = vit_block(pvt_output)
vision_transformer_output = Dense(1, activation='sigmoid', name='vision_transformer_output')(tf.keras.layers.Flatten()(vit_output))

# Define the inputs for the second pipeline
#multimodal_input = Input(shape=multimodal_output.shape[1:], name='multimodal_input')
#distillation_input = Input(shape=distillation_output.shape[1:], name='distillation_input')

input_distillation_second = Input(shape=(distillation_predictions_train.shape[1],), name='input_distillation_second')
input_multimodal_second = Input(shape=(multimodal_predictions_train.shape[1],), name='input_multimodal_second')

#input_distillation_second = Input(shape=(1,), name='input_distillation_second')
#input_multimodal_second = Input(shape=(1,), name='input_multimodal_second')


# Print shapes of inputs before concatenation
print("Shape of input_distillation_second:", input_distillation_second.shape)
print("Shape of input_multimodal_second:", input_multimodal_second.shape)
print("Shape of vision_transformer_output:", vision_transformer_output.shape)


#added new
# Flatten the vision transformer output before concatenation
flattened_vision_output = Flatten()(vision_transformer_output)

# Concatenate the flattened outputs
merged_output = Concatenate()([input_distillation_second,
                               input_multimodal_second,
                               flattened_vision_output])
# Concatenate outputs
#merged_output = Concatenate()([input_distillation_second, input_multimodal_second, vision_transformer_output])

# Flatten the merged output before passing it to the Dense layer
flattened_output = tf.keras.layers.Flatten()(merged_output)

# Final classification head
final_output = Dense(2, activation='softmax', name='output')(flattened_output)

print("Shape of input_mfcc:", tf.shape(input_mfcc))
print("Shape of input_fft:", tf.shape(input_fft))
print("Shape of input_distillation_second:", tf.shape(input_distillation_second))
print("Shape of input_multimodal_second:", tf.shape(input_multimodal_second))

# Create model
model = Model(inputs=[input_mfcc, input_fft, input_distillation_second, input_multimodal_second], outputs=final_output)

# Recompile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

Shape of input_distillation_second: (None, 1)
Shape of input_multimodal_second: (None, 1)
Shape of vision_transformer_output: (None, 1)
Shape of input_mfcc: KerasTensor(type_spec=TensorSpec(shape=(3,), dtype=tf.int32, name=None), inferred_value=[None, 13, 2], name='tf.compat.v1.shape/Shape:0', description="created by layer 'tf.compat.v1.shape'")
Shape of input_fft: KerasTensor(type_spec=TensorSpec(shape=(2,), dtype=tf.int32, name=None), inferred_value=[None, 974], name='tf.compat.v1.shape_1/Shape:0', description="created by layer 'tf.compat.v1.shape_1'")
Shape of input_distillation_second: KerasTensor(type_spec=TensorSpec(shape=(2,), dtype=tf.int32, name=None), inferred_value=[None, 1], name='tf.compat.v1.shape_2/Shape:0', description="created by layer 'tf.compat.v1.shape_2'")
Shape of input_multimodal_second: KerasTensor(type_spec=TensorSpec(shape=(2,), dtype=tf.int32, name=None), inferred_value=[None, 1], name='tf.compat.v1.shape_3/Shape:0', description="created by layer 'tf.compat.v

In [ ]:
#leave this, above one is woekring
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Reshape, Add, MultiHeadAttention, Dropout, Concatenate, Conv1D, Flatten
from tensorflow.keras.models import Model

# Define input shapes for MFCC and FFT features
input_mfcc_shape = (13, 2)  # MFCC shape: 13 coefficients, 2 time steps (assuming 2 channels)
input_fft_shape = (974,)  # FFT shape: 974 features

# Inputs for MFCC and FFT features
input_mfcc = Input(shape=input_mfcc_shape, name='input_mfcc')
input_fft = Input(shape=input_fft_shape, name='input_fft')

# Patch Embedding layer for MFCC and FFT
projection_dim = 256  # Adjust as needed
patch_mfcc = Dense(projection_dim)(Flatten()(input_mfcc))
patch_fft = Dense(projection_dim)(Flatten()(input_fft))

# Concatenate patches
combined_patches = tf.concat([patch_mfcc, patch_fft], axis=1)

# Reshape for PVT block
combined_patches = Reshape((-1, projection_dim))(combined_patches)

# Pyramid Vision Transformer (PVT) block
def pvt_block(x, num_channels):
    shortcut = x
    # Projection layers
    x = Dense(num_channels)(x)
    x = LayerNormalization()(x)
    x = tf.nn.relu(x)
    x = Dense(num_channels)(x)
    x = LayerNormalization()(x)
    x = tf.nn.relu(x)
    # Project the shortcut to match the number of channels
    shortcut = Dense(num_channels * 4)(shortcut)
    x = Dense(num_channels * 4)(x)
    x = LayerNormalization()(x)
    x = Add()([shortcut, x])
    x = tf.nn.relu(x)
    return x

# Apply PVT block
pvt_output = pvt_block(combined_patches, num_channels=256)  # Adjust num_channels as needed

# Vision Transformer (ViT) block
def vit_block(x, num_heads=8, ff_dim=512, dropout_rate=0.1):
    # Multi-head self-attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(attn_output + x)

    # Feed Forward Network
    ffn = Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(out1)
    # Project back to the original dimension
    ffn = Conv1D(filters=projection_dim, kernel_size=1)(ffn)
    ffn = Dropout(dropout_rate)(ffn)

    out2 = LayerNormalization(epsilon=1e-6)(ffn + out1)
    return out2

vit_output = vit_block(pvt_output)
vision_transformer_output = Dense(1, activation='sigmoid', name='vision_transformer_output')(Flatten()(vit_output))

# Define the inputs for the second pipeline
input_distillation_second = Input(shape=(distillation_predictions_train.shape[1],), name='input_distillation_second')
input_multimodal_second = Input(shape=(multimodal_predictions_train.shape[1],), name='input_multimodal_second')

# Concatenate outputs
merged_output = Concatenate()([input_distillation_second, input_multimodal_second, Flatten()(vision_transformer_output)])

# Final classification head
final_output = Dense(2, activation='softmax', name='output')(merged_output)

# Create model
model = Model(inputs=[input_mfcc, input_fft, input_distillation_second, input_multimodal_second], outputs=final_output)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()




In [ ]:
# Your generator function
def combined_generator(image_gen, distillation_data, multimodal_data):
    while True:
        # Get batch from image generator
        try:
            image_data, image_labels = next(image_gen)
        except Exception as e:
            print(f"Error loading image batch: {e}")
            continue  # Skip this batch and move to the next

        # Generate dummy data for MFCC and FFT (replace with actual data)
        batch_size = image_data.shape[0]
        mfcc_data = np.zeros((batch_size, 13, 2))  # Dummy MFCC data
        fft_data = np.zeros((batch_size, 974))     # Dummy FFT data

        # Convert labels to one-hot encoding
        image_labels = tf.keras.utils.to_categorical(image_labels, num_classes=2)

        # Select a batch of distillation and multimodal data
        start_idx = np.random.randint(0, len(distillation_data) - batch_size)
        batch_distillation_data = distillation_data[start_idx: start_idx + batch_size]
        batch_multimodal_data = multimodal_data[start_idx: start_idx + batch_size]

        yield [mfcc_data, fft_data, batch_distillation_data, batch_multimodal_data], image_labels

# Define the base directory for your data
base_dir = '/content/drive/MyDrive/Celeb-DF/Split-Frames'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Define ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Adjust based on your model input size
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

train_steps_per_epoch = train_generator.samples // train_generator.batch_size
val_steps_per_epoch = val_generator.samples // val_generator.batch_size

# Combined training generator
combined_train_gen = combined_generator(train_generator, distillation_predictions_train, multimodal_predictions_train)

# Combined validation generator
combined_val_gen = combined_generator(val_generator, distillation_predictions_test, multimodal_predictions_test)



Found 12571 images belonging to 2 classes.
Found 1541 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [ ]:
# Train the model
model.fit(
    combined_train_gen,
    steps_per_epoch=train_steps_per_epoch,
    epochs=2,  # Adjust based on your requirements
    validation_data=combined_val_gen,
    validation_steps=val_steps_per_epoch
)

Epoch 1/2
325/392 [=======================>......] - ETA: 3:50 - loss: 0.5852 - accuracy: 0.7816

In [ ]:
# Access validation accuracy
val_accuracy = history.history['val_accuracy']
print("Validation Accuracy:", val_accuracy)

In [ ]:
!pip install split-folders

In [ ]:
import os
import shutil
import random

real_frames_path = '/content/drive/MyDrive/Celeb-DF/Frames-Real'
fake_frames_path = '/content/drive/MyDrive/Celeb-DF/Frames-Fake'

output_base_path = '/content/drive/MyDrive/Celeb-DF/Split-Frames'
train_real_path = os.path.join(output_base_path, 'train', 'real')
val_real_path = os.path.join(output_base_path, 'val', 'real')
test_real_path = os.path.join(output_base_path, 'test', 'real')
train_fake_path = os.path.join(output_base_path, 'train', 'fake')
val_fake_path = os.path.join(output_base_path, 'val', 'fake')
test_fake_path = os.path.join(output_base_path, 'test', 'fake')

# Create directories if they don't exist
for folder in [train_real_path, val_real_path, test_real_path, train_fake_path, val_fake_path, test_fake_path]:
    if not os.path.exists(folder):
        os.makedirs(folder)


In [ ]:
def split_and_copy_images(source_folder, train_folder, val_folder, test_folder):
    all_images = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    random.shuffle(all_images)  # Shuffle to ensure randomness

    total_images = len(all_images)
    train_count = int(0.8 * total_images)
    val_count = int(0.1 * total_images)
    test_count = total_images - train_count - val_count  # To ensure all images are included

    train_images = all_images[:train_count]
    val_images = all_images[train_count:train_count + val_count]
    test_images = all_images[train_count + val_count:]

    def copy_images(image_list, source_folder, destination_folder):
        for image in image_list:
            src_path = os.path.join(source_folder, image)
            dst_path = os.path.join(destination_folder, image)
            shutil.copyfile(src_path, dst_path)

    copy_images(train_images, source_folder, train_folder)
    copy_images(val_images, source_folder, val_folder)
    copy_images(test_images, source_folder, test_folder)

In [ ]:
split_and_copy_images(real_frames_path, train_real_path, val_real_path, test_real_path)
split_and_copy_images(fake_frames_path, train_fake_path, val_fake_path, test_fake_path)

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

base_dir = '/content/drive/MyDrive/Celeb-DF/Split-Frames'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:


# Define ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Adjust based on your model input size
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
train_steps_per_epoch = train_generator.samples // train_generator.batch_size
val_steps_per_epoch = val_generator.samples // val_generator.batch_size


Found 12571 images belonging to 2 classes.
Found 1541 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [ ]:
# Assuming you have distillation and multimodal predictions from a previous model
# Get predictions for training data
# predictions_train = modelrgb.predict(X_train)
# distillation_predictions_train = predictions_train[1]  # Access the second element of the list
# multimodal_predictions_train = predictions_train[2]    # Access the third element

# # Get predictions for testing data
# predictions_test = modelrgb.predict(X_test)
# distillation_predictions_test = predictions_test[1]
# multimodal_predictions_test = predictions_test[2]

def combined_generator(image_gen, distillation_data, multimodal_data):
    while True:
        # Get batch from image generator
        image_data, image_labels = next(image_gen)

        # Generate dummy data for MFCC and FFT (replace with actual data)
        batch_size = image_data.shape[0]
        mfcc_data = np.zeros((batch_size, 13, 2))  # Dummy MFCC data
        fft_data = np.zeros((batch_size, 974))     # Dummy FFT data

        # Convert labels to one-hot encoding
        image_labels = tf.keras.utils.to_categorical(image_labels, num_classes=2)

        # Select a batch of distillation and multimodal data
        start_idx = np.random.randint(0, len(distillation_data) - batch_size)
        batch_distillation_data = distillation_data[start_idx: start_idx + batch_size]
        batch_multimodal_data = multimodal_data[start_idx: start_idx + batch_size]

        yield [mfcc_data, fft_data, batch_distillation_data,image_data], image_labels

# Combined training generator
combined_train_gen = combined_generator(train_generator, distillation_predictions_train, multimodal_predictions_train)

# Combined validation generator
combined_val_gen = combined_generator(val_generator, distillation_predictions_test, multimodal_predictions_test)

In [ ]:

# Define batch size for generators
batch_size = 16


def combined_generator(real_gen, fake_gen, distillation_data, multimodal_data):
    while True:
        # Get batches from both generators simultaneously
        real_data, real_labels = next(real_gen)
        fake_data, fake_labels = next(fake_gen)

        # Combine batches
        combined_images = np.concatenate((real_data, fake_data), axis=0)
        combined_labels = np.concatenate((real_labels, fake_labels), axis=0)

        # Shuffle combined batch
        combined_images, combined_labels = shuffle(combined_images, combined_labels, random_state=42)

        # Generate dummy data for MFCC and FFT (replace with actual data)
        batch_size = combined_images.shape[0]
        mfcc_data = np.zeros((batch_size, 13, 2))  # Dummy MFCC data
        fft_data = np.zeros((batch_size, 974))     # Dummy FFT data

        # Convert labels to one-hot encoding
        combined_labels = tf.keras.utils.to_categorical(combined_labels, num_classes=2)

        # Select a batch of distillation and multimodal data
        start_idx = np.random.randint(0, len(distillation_data) - batch_size)
        batch_distillation_data = distillation_data[start_idx: start_idx + batch_size]
        batch_multimodal_data = multimodal_data[start_idx: start_idx + batch_size]

        yield [mfcc_data, fft_data, batch_distillation_data, batch_multimodal_data], combined_labels

# Combined training generator (pass your actual data)
combined_gen = combined_generator(train_generator, train_generator, distillation_predictions_train, multimodal_predictions_train)

# Combined validation generator
validation_gen = combined_generator(val_generator, val_generator, distillation_predictions_test, multimodal_predictions_test)


In [ ]:
# Train the model
model.fit(
    combined_train_gen,
    steps_per_epoch=train_steps_per_epoch,
    epochs=3,  # Adjust based on your requirements
    validation_data=combined_val_gen,
    validation_steps=val_steps_per_epoch
)

MFCC shape: (32, 13, 2)
FFT shape: (32, 974)
Distillation shape: (32, 1)
Image shape: (32, 150, 150, 3)
Epoch 1/3


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/backend.py", line 3580, in concatenate
        return tf.concat([to_dense(x) for x in tensors], axis)

    V

In [ ]:
# from sklearn.model_selection import train_test_split

# mfcc_train_data, mfcc_val_data = train_test_split(input_mfcc, test_size=0.2, random_state=42)
# fft_train_data, fft_val_data = train_test_split(input_fft, test_size=0.2, random_state=42)
# input_distillation_second_train_data, input_distillation_second_val_data = train_test_split(input_distillation_second, test_size=0.2, random_state=42)
# input_multimodal_second_train_data, input_multimodal_second_val_data = train_test_split(input_multimodal_second, test_size=0.2, random_state=42)

TypeError: Expected sequence or array-like, got <class 'keras.src.engine.keras_tensor.KerasTensor'>

In [ ]:
# # Create datasets for the additional inputs
# train_mfcc_dataset = tf.data.Dataset.from_tensor_slices(mfcc_train_data).batch(train_generator.batch_size)
# train_fft_dataset = tf.data.Dataset.from_tensor_slices(fft_train_data).batch(train_generator.batch_size)
# train_distillation_dataset = tf.data.Dataset.from_tensor_slices(input_distillation_second_train_data).batch(train_generator.batch_size)
# train_multimodal_dataset = tf.data.Dataset.from_tensor_slices(input_multimodal_second_train_data).batch(train_generator.batch_size)

# val_mfcc_dataset = tf.data.Dataset.from_tensor_slices(mfcc_val_data).batch(val_generator.batch_size)
# val_fft_dataset = tf.data.Dataset.from_tensor_slices(fft_val_data).batch(val_generator.batch_size)
# val_distillation_dataset = tf.data.Dataset.from_tensor_slices(input_distillation_second_val_data).batch(val_generator.batch_size)  # Assuming you have validation data for these
# val_multimodal_dataset = tf.data.Dataset.from_tensor_slices(input_multimodal_second_val_data).batch(val_generator.batch_size)    # Assuming you have validation data for these


In [ ]:
# history = model.fit(
#     train_generator, input_fft, input_mfcc,
#     steps_per_epoch=train_steps_per_epoch,  # Set based on the combined number of batches for training
#     epochs=2,
#     validation_data=val_generator,
#     validation_steps=val_steps_per_epoch  # Set based on the combined number of batches for validation
# )

NameError: name 'model' is not defined

In [ ]:
# test_steps_per_epoch = test_generator.samples // test_generator.batch_size

# test_loss, test_acc = model.evaluate(
#     test_generator,
#     steps=test_steps_per_epoch
# )

# print(f'Test accuracy: {test_acc}')

In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from sklearn.model_selection import train_test_split
# import os
# import shutil
# import numpy as np
# from sklearn.utils import shuffle

# # Paths to your directories
# real_frames_path = '/content/drive/MyDrive/Celeb-DF/Frames-Real'
# fake_frames_path = '/content/drive/MyDrive/Celeb-DF/Frames-Fake'

# def split_and_copy(src_dir, dst_dirs, split_ratios=(0.7, 0.15, 0.15)):
#     # List all files in the source directory
#     files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]

#     # Shuffle files randomly
#     np.random.shuffle(files)

#     # Calculate number of files for each split
#     num_files = len(files)
#     num_train = int(split_ratios[0] * num_files)
#     num_val = int(split_ratios[1] * num_files)
#     num_test = num_files - num_train - num_val

#     # Assign files to splits
#     train_files = files[:num_train]
#     val_files = files[num_train:num_train + num_val]
#     test_files = files[num_train + num_val:]

#     # Copy files to destination directories
#     for file in train_files:
#         src_path = os.path.join(src_dir, file)
#         dst_path = os.path.join(dst_dirs['train'], file)
#         try:
#             shutil.copy(src_path, dst_path)
#             print(f"Copied {src_path} to {dst_path}")  # Print successful copies
#         except Exception as e:
#             print(f"Error copying {src_path} to {dst_path}: {e}")  # Print errors

#     # Repeat the same process for validation and test files
#     # ... (rest of the function remains the same)

#     # Copy files to destination directories
#     for file in train_files:
#         shutil.copy(os.path.join(src_dir, file), os.path.join(dst_dirs['train'], file))
#     for file in val_files:
#         shutil.copy(os.path.join(src_dir, file), os.path.join(dst_dirs['val'], file))
#     for file in test_files:
#         shutil.copy(os.path.join(src_dir, file), os.path.join(dst_dirs['test'], file))

# # Output paths for splits
# output_base_path = '/content/split'
# output_real_path = os.path.join(output_base_path, 'real')
# output_fake_path = os.path.join(output_base_path, 'fake')

# # Create directories for splits, ensuring they exist
# os.makedirs(os.path.join(output_real_path, 'train'), exist_ok=True)
# os.makedirs(os.path.join(output_real_path, 'val'), exist_ok=True)
# os.makedirs(os.path.join(output_real_path, 'test'), exist_ok=True)

# os.makedirs(os.path.join(output_fake_path, 'train'), exist_ok=True)
# os.makedirs(os.path.join(output_fake_path, 'val'), exist_ok=True)
# os.makedirs(os.path.join(output_fake_path, 'test'), exist_ok=True)

# # ... (rest of your code)

# # Split and copy real frames
# split_and_copy(real_frames_path, {'train': os.path.join(output_real_path, 'train'),
#                                   'val': os.path.join(output_real_path, 'val'),
#                                   'test': os.path.join(output_real_path, 'test')})

# # Split and copy fake frames
# split_and_copy(fake_frames_path, {'train': os.path.join(output_fake_path, 'train'),
#                                   'val': os.path.join(output_fake_path, 'val'),
#                                   'test': os.path.join(output_fake_path, 'test')})

# # Create an ImageDataGenerator instance
# datagen = ImageDataGenerator(rescale=1./255)

# # Define batch size for generators
# batch_size = 16

# # Training Generators
# train_real_generator = datagen.flow_from_directory(
#     directory=os.path.join(output_real_path, 'train'),
#     target_size=(128, 128),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=True
# )

# train_fake_generator = datagen.flow_from_directory(
#     directory=os.path.join(output_fake_path, 'train'),
#     target_size=(128, 128),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=True
# )

# # Validation Generators
# val_real_generator = datagen.flow_from_directory(
#     directory=os.path.join(output_real_path, 'val'),
#     target_size=(128, 128),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=False
# )

# val_fake_generator = datagen.flow_from_directory(
#     directory=os.path.join(output_fake_path, 'val'),
#     target_size=(128, 128),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=False
# )

# # Testing Generators
# test_real_generator = datagen.flow_from_directory(
#     directory=os.path.join(output_real_path, 'test'),
#     target_size=(128, 128),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=False
# )

# test_fake_generator = datagen.flow_from_directory(
#     directory=os.path.join(output_fake_path, 'test'),
#     target_size=(128, 128),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=False
# )

# # Combined generator function
# def combined_generator(real_gen, fake_gen, distillation_data, multimodal_data):
#     while True:
#         # Get batches from both generators simultaneously
#         real_data, real_labels = next(real_gen)
#         fake_data, fake_labels = next(fake_gen)

#         # Combine batches
#         combined_images = np.concatenate((real_data, fake_data), axis=0)
#         combined_labels = np.concatenate((real_labels, fake_labels), axis=0)

#         # Shuffle combined batch
#         combined_images, combined_labels = shuffle(combined_images, combined_labels, random_state=42)

#         # Generate dummy data for MFCC and FFT (replace with actual data)
#         batch_size = combined_images.shape[0]
#         mfcc_data = np.zeros((batch_size, 13, 2))  # Dummy MFCC data
#         fft_data = np.zeros((batch_size, 974))     # Dummy FFT data

#         # Ensure labels are within the valid range [0, num_classes-1]
#         combined_labels = np.clip(combined_labels, 0, 1) # Clip labels to be within [0, 1]

#         # Convert labels to one-hot encoding
#         combined_labels = tf.keras.utils.to_categorical(combined_labels, num_classes=2)

#         # Select a batch of distillation and multimodal data
#         start_idx = np.random.randint(0, len(distillation_data) - batch_size)
#         batch_distillation_data = distillation_data[start_idx: start_idx + batch_size]
#         batch_multimodal_data = multimodal_data[start_idx: start_idx + batch_size]

#         yield [mfcc_data, fft_data, batch_distillation_data, batch_multimodal_data], combined_labels

# # Create combined generators
# combined_gen = combined_generator(train_real_generator, train_fake_generator, distillation_predictions_train, multimodal_predictions_train)
# combined_val_gen = combined_generator(val_real_generator, val_fake_generator, distillation_predictions_train, multimodal_predictions_train)
# combined_test_gen = combined_generator(test_real_generator, test_fake_generator, distillation_predictions_test, multimodal_predictions_test)

In [ ]:
print("Training Real Files:", train_real_generator.filepaths[:1])  # Print first 5 file paths
print("Validation Real Files:", val_real_generator.filepaths[:1])
print("Validation Real Files:", val_fake_generator.filepaths[:3])
print("Validation Real Files:", test_real_generator.filepaths[:3])



Training Real Files: []
Validation Real Files: []
Validation Real Files: []
Validation Real Files: []


In [ ]:
# Determine the number of steps per epoch based on the dataset size and batch size
train_steps_per_epoch = (len(train_real_generator) + len(train_fake_generator)) // batch_size
val_steps_per_epoch = (len(val_real_generator) + len(val_fake_generator)) // batch_size
test_steps_per_epoch = (len(test_real_generator) + len(test_fake_generator)) // batch_size

print("Train steps per epoch:", train_steps_per_epoch)
print("Validation steps per epoch:", val_steps_per_epoch)
print("Test steps per epoch:", test_steps_per_epoch)

In [ ]:
# Train the model using the updated generators
history = model.fit(
    combined_gen,
    steps_per_epoch=train_steps_per_epoch,  # Set based on the combined number of batches for training
    epochs=2,
    validation_data=combined_val_gen,
    validation_steps=val_steps_per_epoch  # Set based on the combined number of batches for validation
)

In [ ]:

# Evaluate the model using the test generator
test_loss, test_acc = model.evaluate(
    combined_test_gen,
    steps=test_steps_per_epoch  # Set based on the combined number of batches for testing
)

print(f"Test accuracy: {test_acc}")

In [ ]:
# # Define paths to your zip files
# real_zip_path = '/content/drive/MyDrive/Celeb-DF/Frames-Real-20240626T130257Z-001.zip'
# fake_zip_path = '/content/drive/MyDrive/Celeb-DF/Frames-Fake-20240626T130258Z-001.zip'

# # Unzip the files
# !unzip -q -o '{real_zip_path}' -d '/content/Frames-Real'
# !unzip -q -o '{fake_zip_path}' -d '/content/Frames-Fake'

# #  --- Data Generators ---
# datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Add validation split

# # Define batch size for generators
# batch_size = 16

# # Training generator
# real_generator = datagen.flow_from_directory(
#     directory='/content/Frames-Real',
#     target_size=(128, 128),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=True,
#     subset='training'  # Use training subset
# )

# fake_generator = datagen.flow_from_directory(
#     directory='/content/Frames-Fake',
#     target_size=(128, 128),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=True,
#     subset='training'  # Use training subset
# )

# # Validation generator
# real_val_generator = datagen.flow_from_directory(
#     directory='/content/Frames-Real',
#     target_size=(128, 128),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=False,  # No need to shuffle validation data
#     subset='validation'  # Use validation subset
# )

# fake_val_generator = datagen.flow_from_directory(
#     directory='/content/Frames-Fake',
#     target_size=(128, 128),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=False,  # No need to shuffle validation data
#     subset='validation'  # Use validation subset
# )

# def combined_generator(real_gen, fake_gen, distillation_data, multimodal_data):
#     while True:
#         # Get batches from both generators simultaneously
#         real_data, real_labels = next(real_gen)
#         fake_data, fake_labels = next(fake_gen)

#         # Combine batches
#         combined_images = np.concatenate((real_data, fake_data), axis=0)
#         combined_labels = np.concatenate((real_labels, fake_labels), axis=0)

#         # Shuffle combined batch
#         combined_images, combined_labels = shuffle(combined_images, combined_labels, random_state=42)

#         # Generate dummy data for MFCC and FFT (replace with actual data)
#         batch_size = combined_images.shape[0]
#         mfcc_data = np.zeros((batch_size, 13, 2))  # Dummy MFCC data
#         fft_data = np.zeros((batch_size, 974))     # Dummy FFT data

#         # Convert labels to one-hot encoding
#         combined_labels = tf.keras.utils.to_categorical(combined_labels, num_classes=2)

#         # Select a batch of distillation and multimodal data
#         start_idx = np.random.randint(0, len(distillation_data) - batch_size)
#         batch_distillation_data = distillation_data[start_idx: start_idx + batch_size]
#         batch_multimodal_data = multimodal_data[start_idx: start_idx + batch_size]

#         yield [mfcc_data, fft_data, batch_distillation_data, batch_multimodal_data], combined_labels

# # Combined training generator (pass your actual data)
# combined_gen = combined_generator(real_generator, fake_generator, distillation_predictions_train, multimodal_predictions_train)

# # Combined validation generator
# validation_gen = combined_generator(real_val_generator, fake_val_generator, distillation_predictions_test, multimodal_predictions_test)


In [ ]:
# # Train the model using the generator
# history = model.fit(
#     combined_gen,
#     steps_per_epoch=200,  # Adjust based on your dataset size
#     epochs=5,
#     validation_data=validation_gen,
#     validation_steps=50  # Adjust based on your validation dataset size
# )

In [ ]:
# import os
# import numpy as np
# from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# # Directory where your test images are stored
# test_data_dir = '/content/drive/MyDrive/Celeb-DF/Frames-YouTube-real'

# # Get list of image files in the directory
# image_files = [f for f in os.listdir(test_data_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# # Initialize lists to store image data and dummy labels (not used for prediction)
# test_images = []
# dummy_labels = []  # Placeholder, won't be used

# # Load and preprocess images
# for image_file in image_files:
#     image_path = os.path.join(test_data_dir, image_file)
#     img = load_img(image_path, target_size=(128, 128))  # Load and resize image
#     img_array = img_to_array(img) / 255.0  # Convert to array and rescale
#     test_images.append(img_array)
#     dummy_labels.append(0)  # Append a dummy label

# # Convert lists to numpy arrays
# test_images = np.array(test_images)
# dummy_labels = np.array(dummy_labels)

# # Generate dummy data for MFCC, FFT, distillation, and multimodal inputs
# num_test_samples = test_images.shape[0]
# mfcc_data = np.zeros((num_test_samples, 13, 2))  # Dummy MFCC data
# fft_data = np.zeros((num_test_samples, 974))     # Dummy FFT data
# distillation_data = np.zeros((num_test_samples, 1))  # Dummy distillation data
# multimodal_data = np.zeros((num_test_samples, 1))    # Dummy multimodal data

# # Make predictions
# predictions = model.predict([mfcc_data, fft_data, distillation_data, multimodal_data])

# # Interpret predictions (assuming binary classification)
# predicted_classes = np.argmax(predictions, axis=1)

# # Print or further process the predicted classes
# print(predicted_classes)

complete

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Reshape, Add, MultiHeadAttention, Dropout, LayerNormalization, Conv1D
from tensorflow.keras.models import Model
import numpy as np

# Define input shapes for MFCC and FFT features
input_mfcc_shape = (13, 2)  # MFCC shape: 13 coefficients, 2 time steps (assuming 2 channels)
input_fft_shape = (974,)  # FFT shape: 974 features

# Inputs for MFCC and FFT features
input_mfcc = Input(shape=input_mfcc_shape, name='input_mfcc')
input_fft = Input(shape=input_fft_shape, name='input_fft')

# Patch Embedding layer for MFCC and FFT
projection_dim = 256  # Adjust as needed
patch_mfcc = Dense(projection_dim)(input_mfcc)
patch_fft = Dense(projection_dim)(input_fft)

# Reshape patch_fft to match the rank of patch_mfcc
patch_fft = Reshape((-1, projection_dim))(patch_fft) # Reshape patch_fft to be 3D

# Concatenate patches
combined_patches = tf.concat([patch_mfcc, patch_fft], axis=1)

# Reshape for PVT block
combined_patches = Reshape((-1, projection_dim))(combined_patches)

# Pyramid Vision Transformer (PVT) block
def pvt_block(x, num_channels):
    shortcut = x
    # Projection layers
    x = Dense(num_channels)(x)
    x = LayerNormalization()(x)
    x = tf.nn.relu(x)
    x = Dense(num_channels)(x)
    x = LayerNormalization()(x)
    x = tf.nn.relu(x)
    # Project the shortcut to match the number of channels
    shortcut = Dense(num_channels * 4)(shortcut)
    x = Dense(num_channels * 4)(x)
    x = LayerNormalization()(x)
    x = Add()([shortcut, x])
    x = tf.nn.relu(x)
    return x

# Apply PVT block
pvt_output = pvt_block(combined_patches, num_channels=256)  # Adjust num_channels as needed

# Vision Transformer (ViT) block
def vit_block(x, num_heads=8, ff_dim=512, dropout_rate=0.1):
    # Multi-head self-attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(attn_output + x)

    # Feed Forward Network
    ffn = Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(out1)
    # Project back to the original dimension
    ffn = Conv1D(filters=projection_dim, kernel_size=1)(ffn)
    ffn = Dropout(dropout_rate)(ffn)

    # Adjust the number of channels in the skip connection to match FFN output
    out1 = Conv1D(filters=projection_dim, kernel_size=1)(out1)

    out2 = LayerNormalization(epsilon=1e-6)(ffn + out1)
    return out2



In [ ]:
# Apply ViT block
vit_output = vit_block(pvt_output)  # Adjust parameters as needed

# Vision Transformer head
# Flatten the output to remove the temporal dimension
vision_transformer_output = Dense(1, activation='sigmoid', name='vision_transformer_output')(tf.keras.layers.Flatten()(vit_output))  # Example for binary classification

# Distillation head
# Flatten the output from the PVT block before passing it to the Dense layer
distillation_output = Dense(10, activation='softmax', name='distillation_output')(tf.keras.layers.Flatten()(pvt_output))  # Example with 10 classes for distillation
# Create model
model = Model(inputs=[input_mfcc, input_fft], outputs=[vision_transformer_output, distillation_output, patch_mfcc, patch_fft, pvt_output])

# Compile model
model.compile(optimizer='adam',
              loss={'vision_transformer_output': 'binary_crossentropy', 'distillation_output': 'categorical_crossentropy'},
              metrics={'vision_transformer_output': 'accuracy', 'distillation_output': 'accuracy'})

# Print model summary
model.summary()

In [ ]:
# Assuming X_mfcc and X_fft are already computed from the frames as shown earlier

# Split the dataset into training and testing sets for frames, MFCC, and FFT
from sklearn.model_selection import train_test_split

# Split frames data
X_frames_train, X_frames_test, _, _ = train_test_split(X, y, test_size=0.2, random_state=42)

# Split MFCC and FFT data
X_mfcc_train, X_mfcc_test, y_mfcc_train, y_mfcc_test = train_test_split(X_mfcc, y, test_size=0.2, random_state=42)
X_fft_train, X_fft_test, y_fft_train, y_fft_test = train_test_split(X_fft, y, test_size=0.2, random_state=42)

# Assuming y_mfcc and y_fft are the same as y for the first pipeline

print(f"Frames Training data shape: {X_frames_train.shape}")
print(f"Frames Testing data shape: {X_frames_test.shape}")
print(f"MFCC Training data shape: {X_mfcc_train.shape}")
print(f"MFCC Testing data shape: {X_mfcc_test.shape}")
print(f"FFT Training data shape: {X_fft_train.shape}")
print(f"FFT Testing data shape: {X_fft_test.shape}")

In [ ]:
from keras.utils import to_categorical
# Convert labels to categorical (assuming 10 classes for distillation)
y_train_categories = to_categorical(y_train, num_classes=10)
y_test_categories = to_categorical(y_test, num_classes=10)


# Define callbacks if needed (e.g., for early stopping, checkpointing)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True),
    # Add more callbacks as needed
]

# Compile model (if not compiled already)
model.compile(optimizer='adam',
              loss={'vision_transformer_output': 'binary_crossentropy', 'distillation_output': 'categorical_crossentropy'},
              metrics={'vision_transformer_output': 'accuracy', 'distillation_output': 'accuracy'})

# Train the model
history = model.fit(
    {'input_mfcc': X_mfcc_train, 'input_fft': X_fft_train},
    {'vision_transformer_output': y_train, 'distillation_output': y_train_categories},
    epochs=10,  # Adjust number of epochs as needed
    batch_size=32,  # Adjust batch size based on your hardware capabilities
    validation_data=(
        {'input_mfcc': X_mfcc_test, 'input_fft': X_fft_test},
        {'vision_transformer_output': y_test, 'distillation_output': y_test_categories}
    ),
    callbacks=callbacks
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Reshape, Add, MultiHeadAttention, Dropout, LayerNormalization, Conv1D
from tensorflow.keras.models import Model
import numpy as np

# Define input shapes for MFCC and FFT features
input_mfcc_shape = (13, 2)  # MFCC shape: 13 coefficients, 2 time steps (assuming 2 channels)
input_fft_shape = (974,)  # FFT shape: 974 features

# Inputs for MFCC and FFT features
input_mfcc = Input(shape=input_mfcc_shape, name='input_mfcc')
input_fft = Input(shape=input_fft_shape, name='input_fft')

# Patch Embedding layer for MFCC and FFT
projection_dim = 256  # Adjust as needed
patch_mfcc = Dense(projection_dim)(input_mfcc)
patch_fft = Dense(projection_dim)(input_fft)

# Reshape patch_fft to match the rank of patch_mfcc
patch_fft = Reshape((-1, projection_dim))(patch_fft) # Reshape patch_fft to be 3D

# Concatenate patches
combined_patches = tf.concat([patch_mfcc, patch_fft], axis=1)

# Reshape for PVT block
combined_patches = Reshape((-1, projection_dim))(combined_patches)

# Pyramid Vision Transformer (PVT) block
def pvt_block(x, num_channels):
    shortcut = x
    # Projection layers
    x = Dense(num_channels)(x)
    x = LayerNormalization()(x)
    x = tf.nn.relu(x)
    x = Dense(num_channels)(x)
    x = LayerNormalization()(x)
    x = tf.nn.relu(x)
    # Project the shortcut to match the number of channels
    shortcut = Dense(num_channels * 4)(shortcut)
    x = Dense(num_channels * 4)(x)
    x = LayerNormalization()(x)
    x = Add()([shortcut, x])
    x = tf.nn.relu(x)
    return x

# Apply PVT block
pvt_output = pvt_block(combined_patches, num_channels=256)  # Adjust num_channels as needed

# Vision Transformer (ViT) block
def vit_block(x, num_heads=8, ff_dim=512, dropout_rate=0.1):
    # Multi-head self-attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(attn_output + x)

    # Feed Forward Network
    ffn = Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(out1)
    # Project back to the original dimension
    ffn = Conv1D(filters=projection_dim, kernel_size=1)(ffn)
    ffn = Dropout(dropout_rate)(ffn)

    # Adjust the number of channels in the skip connection to match FFN output
    out1 = Conv1D(filters=projection_dim, kernel_size=1)(out1)

    out2 = LayerNormalization(epsilon=1e-6)(ffn + out1)
    return out2



In [ ]:
vit_output = vit_block(pvt_output)
vision_transformer_output = Dense(1, activation='sigmoid', name='vision_transformer_output')(tf.keras.layers.Flatten()(vit_output))

input_distillation_second = Input(shape=(1,), name='input_distillation_second')
input_multimodal_second = Input(shape=(1,), name='input_multimodal_second')

# Concatenate outputs (use the new input layer names)
merged_output = Concatenate()([input_distillation_second, input_multimodal_second, vision_transformer_output])

# Final classification head
final_output = Dense(1, activation='sigmoid', name='output')(merged_output)

distillation_predictions_train = tf.convert_to_tensor(distillation_predictions_train)
multimodal_predictions_train = tf.convert_to_tensor(multimodal_predictions_train)

#create model
model = Model(inputs=[input_mfcc, input_fft, input_distillation_second, input_multimodal_second],
              outputs=final_output)

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Example loss function, adjust as needed
              metrics=['accuracy'])         # Example metrics, adjust as needed

# Display model summary
model.summary()


In [ ]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Define paths to your zip files
real_zip_path = '/content/drive/MyDrive/Celeb-DF/Frames-Real-20240626T130257Z-001.zip'
fake_zip_path = '/content/drive/MyDrive/Celeb-DF/Frames-Fake-20240626T130258Z-001.zip'

# Unzip the files
!unzip -q -o '{real_zip_path}' -d '/content/Frames-Real'
!unzip -q -o '{fake_zip_path}' -d '/content/Frames-Fake'

#  --- Data Generators ---
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Add validation split

# Define batch size for generators
batch_size = 16

# Training generator
real_generator = datagen.flow_from_directory(
    directory='/content/Frames-Real',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    subset='training'  # Use training subset
)

fake_generator = datagen.flow_from_directory(
    directory='/content/Frames-Fake',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    subset='training'  # Use training subset
)

# Validation generator
real_val_generator = datagen.flow_from_directory(
    directory='/content/Frames-Real',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,  # No need to shuffle validation data
    subset='validation'  # Use validation subset
)

fake_val_generator = datagen.flow_from_directory(
    directory='/content/Frames-Fake',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,  # No need to shuffle validation data
    subset='validation'  # Use validation subset
)


def combined_generator(real_gen, fake_gen):
    while True:
        # Get batches from both generators simultaneously
        real_data, real_labels = next(real_gen)
        fake_data, fake_labels = next(fake_gen)

        # Combine batches
        combined_images = np.concatenate((real_data, fake_data), axis=0)
        combined_labels = np.concatenate((real_labels, fake_labels), axis=0)

        # Shuffle combined batch
        combined_images, combined_labels = shuffle(combined_images, combined_labels, random_state=42)

        # Generate dummy data for MFCC and FFT (replace with actual data)
        batch_size = combined_images.shape[0]
        mfcc_data = np.zeros((batch_size, 13, 130))  # Dummy MFCC data
        fft_data = np.zeros((batch_size, 130))       # Dummy FFT data

        # Convert labels to one-hot encoding
        combined_labels = tf.keras.utils.to_categorical(combined_labels, num_classes=2)

        yield [mfcc_data, fft_data, combined_images], combined_labels

# Combined training generator
combined_gen = combined_generator(real_generator, fake_generator)

# Combined validation generator
validation_gen = combined_generator(real_val_generator, fake_val_generator)

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming X_mfcc, X_fft, and y are defined
X_mfcc_train, X_mfcc_test, y_train, y_test = train_test_split(X_mfcc, y, test_size=0.2, random_state=42)
X_fft_train, X_fft_test, _, _ = train_test_split(X_fft, y, test_size=0.2, random_state=42)

# Distillation and multimodal outputs should not be split; they come directly from the first pipeline
distillation_train = np.array(distillation_predictions_train)  # Ensure they are numpy arrays
multimodal_train = np.array(multimodal_predictions_train)


In [ ]:
from keras.utils import to_categorical

# Convert labels to categorical (binary classification in this case)
y_train_categories = to_categorical(y_train, num_classes=2)
y_test_categories = to_categorical(y_test, num_classes=2)

# One-hot encode distillation and multimodal outputs (if they are not already)
distillation_train = to_categorical(distillation_predictions_train, num_classes=2)
multimodal_train = to_categorical(multimodal_predictions_train, num_classes=2)


# Squeeze the distillation and multimodal outputs to remove the extra dimension from one-hot encoding
distillation_train = np.argmax(distillation_train, axis=1)
multimodal_train = np.argmax(multimodal_train, axis=1)

# Ensure that distillation_train and multimodal_train have the same length as your training data
distillation_train = distillation_train[:len(X_mfcc_train)]
multimodal_train = multimodal_train[:len(X_mfcc_train)]

# Squeeze the distillation and multimodal outputs to remove the extra dimension from one-hot encoding
merged_output = Concatenate()([input_distillation_second, input_multimodal_second, vision_transformer_output])

# Final classification head (keep 2 output neurons for binary classification)
final_output = Dense(2, activation='softmax', name='output')(merged_output)

#create model (UPDATED)
model = Model(inputs=[input_mfcc, input_fft, input_distillation_second, input_multimodal_second],
              outputs=final_output)

# Compile the model with the categorical_crossentropy loss function
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Make sure that X_mfcc_test, X_fft_test have the same number of samples as
# distillation_train and multimodal_train in validation data
X_mfcc_test = X_mfcc_test[:len(distillation_train)]
X_fft_test = X_fft_test[:len(distillation_train)]
y_test_categories = y_test_categories[:len(distillation_train)]

# Train the model
history = model.fit(
    {'input_mfcc': X_mfcc_train, 'input_fft': X_fft_train,
     'input_distillation_second': distillation_train, 'input_multimodal_second': multimodal_train},
    y_train_categories,
    epochs=5,
    batch_size=16,
    validation_data=(
        {'input_mfcc': X_mfcc_test, 'input_fft': X_fft_test,
         'input_distillation_second': distillation_train, 'input_multimodal_second': multimodal_train},
        y_test_categories
    )
)

DUMMY

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Reshape, Add, MultiHeadAttention, Dropout, Concatenate, Conv1D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np

# Define input shapes for MFCC and FFT features
input_mfcc_shape = (13, 2)
input_fft_shape = (974,)

# Inputs for MFCC and FFT features
input_mfcc = Input(shape=input_mfcc_shape, name='input_mfcc')
input_fft = Input(shape=input_fft_shape, name='input_fft')

# Patch Embedding layer for MFCC and FFT
projection_dim = 256
patch_mfcc = Dense(projection_dim)(input_mfcc)
patch_fft = Dense(projection_dim)(input_fft)

# Reshape patch_fft to match the rank of patch_mfcc
patch_fft = Reshape((-1, projection_dim))(patch_fft)

# Concatenate patches
combined_patches = tf.concat([patch_mfcc, patch_fft], axis=1)

# Pyramid Vision Transformer (PVT) block
def pvt_block(x, num_channels):
    shortcut = x
    x = Dense(num_channels)(x)
    x = LayerNormalization()(x)
    x = tf.nn.relu(x)
    x = Dense(num_channels)(x)
    x = LayerNormalization()(x)
    x = tf.nn.relu(x)
    shortcut = Dense(num_channels * 4)(shortcut)
    x = Dense(num_channels * 4)(x)
    x = LayerNormalization()(x)
    x = Add()([shortcut, x])
    x = tf.nn.relu(x)
    return x

# Apply PVT block
pvt_output = pvt_block(combined_patches, num_channels=256)

# Vision Transformer (ViT) block
def vit_block(x, num_heads=8, ff_dim=512, dropout_rate=0.1):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(attn_output + x)
    ffn = Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(out1)
    ffn = Conv1D(filters=projection_dim, kernel_size=1)(ffn)
    ffn = Dropout(dropout_rate)(ffn)
    out1 = Conv1D(filters=projection_dim, kernel_size=1)(out1)
    out2 = LayerNormalization(epsilon=1e-6)(ffn + out1)
    return out2

vit_output = vit_block(pvt_output)
vision_transformer_output = Dense(1, activation='sigmoid', name='vision_transformer_output')(tf.keras.layers.Flatten()(vit_output))

input_distillation_second = Input(shape=(1,), name='input_distillation_second')
input_multimodal_second = Input(shape=(1,), name='input_multimodal_second')

merged_output = Concatenate()([input_distillation_second, input_multimodal_second, vision_transformer_output])

# Final classification head
final_output = Dense(2, activation='softmax', name='output')(merged_output)

# Create model
model = Model(inputs=[input_mfcc, input_fft, input_distillation_second, input_multimodal_second], outputs=final_output)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Generate dummy data for MFCC and FFT
X_mfcc = np.random.rand(15426, 13, 2)
X_fft = np.random.rand(15426, 974)
y = np.random.randint(2, size=15426)
distillation_predictions_train = np.random.randint(2, size=15426)
multimodal_predictions_train = np.random.randint(2, size=15426)

# Split data
X_mfcc_train, X_mfcc_test, y_train, y_test = train_test_split(X_mfcc, y, test_size=0.2, random_state=42)
X_fft_train, X_fft_test, _, _ = train_test_split(X_fft, y, test_size=0.2, random_state=42)
distillation_train = distillation_predictions_train[:len(X_mfcc_train)]
multimodal_train = multimodal_predictions_train[:len(X_mfcc_train)]

# Convert labels to categorical
y_train_categories = to_categorical(y_train, num_classes=2)
y_test_categories = to_categorical(y_test, num_classes=2)

# Convert distillation and multimodal outputs to categorical
distillation_train = to_categorical(distillation_train, num_classes=2)
multimodal_train = to_categorical(multimodal_train, num_classes=2)

# Train the model
history = model.fit(
    {'input_mfcc': X_mfcc_train, 'input_fft': X_fft_train,
     'input_distillation_second': distillation_train[:, 1], 'input_multimodal_second': multimodal_train[:, 1]},
    y_train_categories,
    epochs=5,
    batch_size=16,
    validation_data=(
        {'input_mfcc': X_mfcc_test, 'input_fft': X_fft_test,
         'input_distillation_second': distillation_train[:len(X_mfcc_test), 1], 'input_multimodal_second': multimodal_train[:len(X_mfcc_test), 1]},
        y_test_categories
    )
)


In [ ]:
def combined_generator(real_gen, fake_gen, X_mfcc, X_fft, distillation, multimodal, y):
    real_index = 0
    fake_index = 0

    while True:
        # Get batches from both generators simultaneously
        real_data, _ = next(real_gen)
        fake_data, _ = next(fake_gen)

        # Combine batches
        combined_images = np.concatenate((real_data, fake_data), axis=0)
        batch_size = combined_images.shape[0]

        # Calculate the number of samples to take from each half
        num_real = real_data.shape[0]
        num_fake = fake_data.shape[0]


        # Extract corresponding MFCC, FFT, distillation, multimodal, and labels
        # Ensure consistent slicing for all data types
        mfcc_data = np.concatenate((
            X_mfcc[real_index : real_index + num_real],
            X_mfcc[len(X_mfcc)//2 + fake_index : len(X_mfcc)//2 + fake_index + num_fake]
        ))
        fft_data = np.concatenate((
            X_fft[real_index : real_index + num_real],
            X_fft[len(X_fft)//2 + fake_index : len(X_fft)//2 + fake_index + num_fake]
        ))
        distillation_data = np.concatenate((
            distillation[real_index : real_index + num_real],
            distillation[len(distillation)//2 + fake_index : len(distillation)//2 + fake_index + num_fake]
        ))
        multimodal_data = np.concatenate((
            multimodal[real_index : real_index + num_real],
            multimodal[len(multimodal)//2 + fake_index : len(multimodal)//2 + fake_index + num_fake]
        ))
        combined_labels = np.concatenate((
            y[real_index : real_index + num_real],
            y[len(y)//2 + fake_index : len(y)//2 + fake_index + num_fake]
        ))

        # Update indices and reset if necessary (NEW CODE)
        real_index = (real_index + num_real) % len(X_mfcc)
        fake_index = (fake_index + num_fake) % len(X_mfcc)

        print("Index update logic:")
        print("  real_index =", real_index, "(calculated as (", real_index - num_real, "+", num_real, ") %", len(X_mfcc), ")")
        print("  fake_index =", fake_index, "(calculated as (", fake_index - num_fake, "+", num_fake, ") %", len(X_mfcc), ")")

        # Shuffle indices instead of arrays directly
        indices = np.arange(batch_size)
        np.random.shuffle(indices)

        # Apply shuffling to all arrays using the shuffled indices
        combined_images = combined_images[indices]
        mfcc_data = mfcc_data[indices]
        fft_data = fft_data[indices]
        distillation_data = distillation_data[indices]
        multimodal_data = multimodal_data[indices]
        combined_labels = combined_labels[indices]

        print("Real index:", real_index)
        print("Fake index:", fake_index)
        print("Num real:", num_real)
        print("Num fake:", num_fake)
        print("Shapes:", [arr.shape for arr in [mfcc_data, fft_data, distillation_data, multimodal_data, combined_labels]])

        yield [mfcc_data, fft_data, distillation_data, multimodal_data], combined_labels

In [ ]:
print("X_mfcc_train shape:", X_mfcc_train.shape)
print("X_fft_train shape:", X_fft_train.shape)
print("distillation_train shape:", distillation_train.shape)
print("multimodal_train shape:", multimodal_train.shape)
print("y_train_categories shape:", y_train_categories.shape)
print("y shape:", y.shape)

In [ ]:
# Create combined generators (pass precomputed data)
combined_gen = combined_generator(
    real_generator, fake_generator, X_mfcc_train, X_fft_train,
    distillation_train, multimodal_train, y_train_categories
)

validation_gen = combined_generator(
    real_val_generator, fake_val_generator, X_mfcc_test, X_fft_test,
    distillation_train, multimodal_train, y_test_categories  # Use appropriate validation data
)

# Iterate over the generator for a few steps to see the print output
for _ in range(3):  # Adjust the number of steps as needed
    data, labels = next(combined_gen)
    print("Data shapes:", [arr.shape for arr in data], "Labels shape:", labels.shape)

In [ ]:

# Train the model using the combined generators
history = model.fit(
    combined_gen,
    epochs=10,
    steps_per_epoch=len(real_generator),
    validation_data=validation_gen,
    validation_steps=len(real_val_generator)
)

In [ ]:
# Distillation and Multimodal Outputs from First Pipeline - Assuming placeholders
# Replace with actual outputs from your first pipeline
distillation_output_from_pipeline1 = Input(shape=(1,), name='distillation_output')
multimodal_output_from_pipeline1 = Input(shape=(1,), name='multimodal_output')

# Recreate necessary layers for the new model
input_mfcc_new = Input(shape=input_mfcc_shape, name='input_mfcc_new')
input_fft_new = Input(shape=input_fft_shape, name='input_fft_new')

projection_dim = 256
patch_mfcc_new = Dense(projection_dim)(input_mfcc_new)
patch_fft_new = Dense(projection_dim)(input_fft_new)
patch_fft_new = Reshape((-1, projection_dim))(patch_fft_new)
combined_patches_new = tf.concat([patch_mfcc_new, patch_fft_new], axis=1)
combined_patches_new = Reshape((-1, projection_dim))(combined_patches_new)
pvt_output_new = pvt_block(combined_patches_new, num_channels=256)
vit_output_new = vit_block(pvt_output_new)
vision_transformer_output_new = Dense(1, activation='sigmoid', name='vision_transformer_output')(tf.keras.layers.Flatten()(vit_output_new))

# Concatenate outputs from first pipeline and vision transformer output
merged_output = Concatenate()([distillation_output_from_pipeline1, multimodal_output_from_pipeline1, vision_transformer_output_new])

# Final classification head
final_output = Dense(1, activation='sigmoid', name='output')(merged_output)

# Create model
model = Model(inputs=[input_mfcc_new, input_fft_new, distillation_output_from_pipeline1, multimodal_output_from_pipeline1],
              outputs=final_output)

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Example loss function, adjust as needed
              metrics=['accuracy'])         # Example metrics, adjust as needed

# Display model summary
model.summary()

In [ ]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Define paths to your zip files
real_zip_path = '/content/drive/MyDrive/Celeb-DF/Frames-Real-20240626T130257Z-001.zip'
fake_zip_path = '/content/drive/MyDrive/Celeb-DF/Frames-Fake-20240626T130258Z-001.zip'

# Unzip the files
!unzip -q -o '{real_zip_path}' -d '/content/Frames-Real'
!unzip -q -o '{fake_zip_path}' -d '/content/Frames-Fake'

#  --- Data Generators ---
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Add validation split

# Define batch size for generators
batch_size = 16

# Training generator
real_generator = datagen.flow_from_directory(
    directory='/content/Frames-Real',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    subset='training'  # Use training subset
)

fake_generator = datagen.flow_from_directory(
    directory='/content/Frames-Fake',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    subset='training'  # Use training subset
)

# Validation generator
real_val_generator = datagen.flow_from_directory(
    directory='/content/Frames-Real',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,  # No need to shuffle validation data
    subset='validation'  # Use validation subset
)

fake_val_generator = datagen.flow_from_directory(
    directory='/content/Frames-Fake',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,  # No need to shuffle validation data
    subset='validation'  # Use validation subset
)


def combined_generator(real_gen, fake_gen):
    while True:
        # Get batches from both generators simultaneously
        real_data, real_labels = next(real_gen)
        fake_data, fake_labels = next(fake_gen)

        # Combine batches
        combined_images = np.concatenate((real_data, fake_data), axis=0)
        combined_labels = np.concatenate((real_labels, fake_labels), axis=0)

        # Shuffle combined batch
        combined_images, combined_labels = shuffle(combined_images, combined_labels, random_state=42)

        # Generate dummy data for MFCC and FFT (replace with actual data)
        batch_size = combined_images.shape[0]
        mfcc_data = np.zeros((batch_size, 13, 130))  # Dummy MFCC data
        fft_data = np.zeros((batch_size, 130))       # Dummy FFT data

        # Convert labels to one-hot encoding
        combined_labels = tf.keras.utils.to_categorical(combined_labels, num_classes=2)

        yield [mfcc_data, fft_data, combined_images], combined_labels

# Combined training generator
combined_gen = combined_generator(real_generator, fake_generator)

# Combined validation generator
validation_gen = combined_generator(real_val_generator, fake_val_generator)

In [ ]:
#Get a sample batch from the generator to determine input shape
sample_batch1 = next(combined_gen)
# Print shapes of individual elements within sample_batch1[0] for debugging
print([np.array(data).shape for data in sample_batch1[0]])

# Assuming you want the shape of the image data (third element), proceed as follows:
input_shape_gen = np.array(sample_batch1[0][2]).shape[1:] # Select the third element (index 2), convert to NumPy array, and remove batch dimension

#Get a sample batch from the generator to determine input shape
sample_batch2 = next(validation_gen)
# Print shapes of individual elements within sample_batch1[0] for debugging
print([np.array(data).shape for data in sample_batch2[0]])

# Assuming you want the shape of the image data (third element), proceed as follows:
input_shape_gen = np.array(sample_batch2[0][2]).shape[1:] # Select the third element (index 2), convert to NumPy array, and remove batch dimension


# Now define input layers with the correct shapes
input_mfcc = Input(shape=input_mfcc_new.shape[1:], name='input_mfcc')  # Use shape from previous input layer
input_fft = Input(shape=input_fft_new.shape[1:], name='input_fft')    # Use shape from previous input layer
input_gen = Input(shape=input_shape_gen[1:], name='input_gen')            # Use shape from generator sample, removing batch dimension

input_combined = Input(shape=merged_output.shape[1:], name='input_combined') # Use shape from previous layer, removing batch dimension

In [ ]:
# Assuming 'pvt_block' and 'vit_block' are defined elsewhere and accessible

# Use the NEW input layers you defined for this model
projection_dim = 256
# Use input_mfcc, NOT input_mfcc_new
patch_mfcc_new = Dense(projection_dim)(input_mfcc)
# Use input_fft, NOT input_fft_new
patch_fft_new = Dense(projection_dim)(input_fft)
patch_fft_new = Reshape((-1, projection_dim))(patch_fft_new)
combined_patches_new = tf.concat([patch_mfcc_new, patch_fft_new], axis=1)
combined_patches_new = Reshape((-1, projection_dim))(combined_patches_new)
pvt_output_new = pvt_block(combined_patches_new, num_channels=256)
vit_output_new = vit_block(pvt_output_new)
vision_transformer_output_new = Dense(1, activation='sigmoid', name='vision_transformer_output')(tf.keras.layers.Flatten()(vit_output_new))

# Concatenate outputs from first pipeline and vision transformer output
# Use the placeholder tensors for outputs from the first pipeline
merged_output = Concatenate()([distillation_output_from_pipeline1, multimodal_output_from_pipeline1, vision_transformer_output_new])

# Final classification head
final_output = Dense(1, activation='sigmoid', name='output')(merged_output)

# Create model with the specified inputs
# Use the new input layers you defined
model = Model(inputs=[input_mfcc, input_fft, input_gen,
                      distillation_output_from_pipeline1, multimodal_output_from_pipeline1],
              outputs=final_output)

# Compile model (adjust loss and metrics as needed)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(
    combined_gen, # Pass the generator directly
    epochs=10,
    steps_per_epoch=len(real_generator),
    validation_data=validation_gen,
    validation_steps=len(real_val_generator),
    callbacks=callbacks
)